## 0. Introduction -
Exploratory Data Analysis (EDA)




This notebook analyzes customer behavior to understand **why customers leave**
a telecom company using advanced interactive visualizations.

Focus areas:
- Customer tenure
- Monthly and total charges
- Contract type
- Internet and service features
- Payment methods

The goal is to discover actionable patterns that help reduce churn.


In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio



In [16]:
# -------- GLOBAL DARK THEME --------
pio.templates["elegant_dark"] = {
    "layout": {
        "paper_bgcolor": "#0e1117",
        "plot_bgcolor": "#0e1117",
        "font": {"color": "white", "family": "Arial"},
        "title": {"font": {"size": 20, "color": "white"}, "x": 0.5},
        "xaxis": {"showgrid": False, "tickcolor": "white", "linecolor": "white"},
        "yaxis": {"showgrid": False, "tickcolor": "white", "linecolor": "white"},
        "legend": {"font": {"color": "white"}}
    }
}
pio.templates.default = "elegant_dark"

# -------- COLOR PALETTE --------
COLORS = {
    "Yes": "#d2878c",   # churn
    "No":  "#6f9ea3"    # stay
}


In [18]:
df = pd.read_csv("../data/cleaned_telecom_churn_data.csv")
df.head()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,1,0,0,0,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,0
2,1,0,0,0,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,1,0,0,0,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,0,0,0,0,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1


In [19]:
# Convert churn to Yes/No if needed
if df["Churn"].dtype != "O":
    df["Churn"] = df["Churn"].map({0: "No", 1: "Yes"})

df["Churn"].value_counts()



Churn
No     5163
Yes    1869
Name: count, dtype: int64

In [20]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   int64  
 1   SeniorCitizen     7032 non-null   int64  
 2   Partner           7032 non-null   int64  
 3   Dependents        7032 non-null   int64  
 4   tenure            7032 non-null   int64  
 5   PhoneService      7032 non-null   int64  
 6   MultipleLines     7032 non-null   object 
 7   InternetService   7032 non-null   object 
 8   OnlineSecurity    7032 non-null   object 
 9   OnlineBackup      7032 non-null   object 
 10  DeviceProtection  7032 non-null   object 
 11  TechSupport       7032 non-null   object 
 12  StreamingTV       7032 non-null   object 
 13  StreamingMovies   7032 non-null   object 
 14  Contract          7032 non-null   object 
 15  PaperlessBilling  7032 non-null   int64  
 16  PaymentMethod     7032 non-null   object 


In [21]:
df.describe()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges
count,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000,7032.000000
mean,0.504693,0.162400,0.482509,0.298493,32.421786,0.903299,0.592719,64.798208,2283.300441
std,0.500014,0.368844,0.499729,0.457629,24.545260,0.295571,0.491363,30.085974,2266.771362
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,18.250000,18.800000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.587500,401.450000
50%,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.350000,1397.475000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.862500,3794.737500
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,8684.800000


# Dataset Overview

The cleaned dataset contains:
  A churn flag Churn: Yes/No
  Gender and senior citizenship
  Contract and payment details
  Multiple service features phone, internet, security, streaming
  Numeric features like tenure, MonthlyCharges, TotalCharges

Next, we explore how these features relate to churn using visualizations.


### Churn Distribution

In [22]:
fig = px.pie(
    df,
    names="Churn",
    hole=0.6,
    color="Churn",
    color_discrete_map=COLORS,
    title="Overall Churn Distribution"
)

fig.update_traces(textinfo="percent+label", pull=[0, 0.06])
fig.show()


-Customer churn represents a serious business risk.
Even small percentages can mean major financial loss at scale.


### TENURE VS CHURN

In [23]:
fig = px.histogram(
    df,
    x="tenure",
    color="Churn",
    nbins=30,
    barmode="overlay",
    opacity=0.8,
    color_discrete_map=COLORS,
    title="Tenure Distribution by Churn"
)
fig.update_layout(xaxis_title="Tenure (months)", yaxis_title="Customer Count")
fig.show()


New customers churn much more than long-term customers.
Retention should focus heavily on the first months.

Business implication:
Early customer experience is crucial.
Onboarding quality directly affects retention.

### Monthly Charges 

In [35]:
fig = px.strip(
    df,
    x="Churn",
    y="MonthlyCharges",
    color="Churn",
    color_discrete_map=COLORS,
    title="Monthly Charges Distribution by Churn"
)

fig.update_traces(jitter=0.3, opacity=0.6)
fig.update_layout(
    yaxis_title="Monthly Charges ($)",
    xaxis_title="Churn",
)
fig.show()



### Insight — Pricing Strategy

Customers who churn typically pay more per month.
Pricing plays a major role in dissatisfaction and churn behavior.
Managers should focus on pricing fairness and bundles.


## Contract Type vs Churn

In [26]:
fig = px.histogram(
    df,
    x="Contract",
    color="Churn",
    barmode="group",
    color_discrete_map=COLORS,
    title="Contract Type vs Churn"
)
fig.show()


### Insight — Contract Stability

Month-to-month contracts have the highest churn.
One-year and two-year contracts are much more stable.

Recommendation:
Incentivize long-term subscriptions.


## CONTRACT TYPE VS CHURN (RATE %)

In [31]:
color_map = {
    "Month-to-month": "#d2878c",  # soft coral
    "One year": "#9b8db3",        # muted purple
    "Two year": "#6f9ea3"         # soft teal
}

fig = px.bar(
    contract_rate,
    x="Contract",
    y="ChurnRate",
    text="ChurnRate",
    color="Contract",
    color_discrete_map=color_map,
    title="Churn Rate by Contract Type (%)",
    labels={"ChurnRate": "Churn (%)"}
)

# White labels and values
fig.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='outside',
    textfont=dict(color="white", size=13)
)

# Dark mode layout fix
fig.update_layout(
    title_font_color="white",
    font=dict(color="white", size=12),
    plot_bgcolor="#0e1117",
    paper_bgcolor="#0e1117",
    showlegend=False,
    margin=dict(l=40, r=20, t=60, b=40)
)

fig.update_xaxes(showgrid=False, color="white")
fig.update_yaxes(showgrid=False, color="white")

fig.show()

Month-to-month contracts are the most unstable.
Long-term plans drastically reduce churn.

Contract locks reduce churn risk significantly.


## Internet Service vs Churn

In [28]:
fig = px.histogram(
    df,
    x="InternetService",
    color="Churn",
    barmode="group",
    color_discrete_map=COLORS,
    title="Internet Service vs Churn"
)
fig.show()


## Payment Method vs Churn

In [29]:
fig = px.histogram(
    df,
    x="PaymentMethod",
    color="Churn",
    barmode="group",
    color_discrete_map=COLORS,
    title="Payment Method vs Churn"
)

fig.update_layout(xaxis_tickangle=-30)
fig.show()


## RISK ZONES HEATMAP

In [43]:
heat = df.copy()

# Create bins with readable labels
tenure_bins = pd.cut(heat["tenure"], bins=4)
charge_bins = pd.cut(heat["MonthlyCharges"], bins=5)

def format_bin(interval, unit=""):
    return f"{int(interval.left)}–{int(interval.right)}{unit}"

heat["TenureReadable"] = tenure_bins.apply(lambda x: format_bin(x, " months"))
heat["ChargeReadable"] = charge_bins.apply(lambda x: format_bin(x, "$"))

# Convert Churn to numeric
heat["ChurnVal"] = heat["Churn"].map({"Yes": 1, "No": 0})

# Create pivot table with readable labels
pivot = heat.pivot_table(
    index="TenureReadable",
    columns="ChargeReadable",
    values="ChurnVal",
    aggfunc="mean"
)

# Plot heatmap
fig = px.imshow(
    pivot,
    text_auto=".0%",
    color_continuous_scale=["#0e1117", COLORS["Yes"]],
    title="Churn Risk Zones by Tenure and Price",
    aspect="auto"
)

# Layout polish
fig.update_layout(
    xaxis_title="Monthly Charges Range ($)",
    yaxis_title="Tenure Range (Months)",
    title_font_size=20,
    width=500,
    height=550 
)
fig.update_yaxes(
    tickangle=0,
    automargin=True
)

fig.show()


C:\Users\User\AppData\Local\Temp\ipykernel_23780\63787708.py:17: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior




This heatmap highlights where churn risk is highest based on how long customers have stayed and how much they pay each month. Lighter areas indicate higher churn probability, and the percentages inside each box represent the churn rate for customers in that group. The visualization clearly shows that customers who are newer and paying higher monthly charges are much more likely to leave than long-term or lower-paying customers. This suggests that pricing pressure and early customer experience are key drivers of churn, and that retention efforts should focus on supporting new customers and offering better value to those on expensive plans.



# Business Summary

Key findings from EDA:

✔ Customers leave early in their lifecycle → Improve onboard 
✔ High monthly charges increase churn  
✔ Month-to-month contracts are risky  → Push yearly plans
✔ Long-term contracts increase stability  
✔ Pricing strategy directly affects retention  → Monitor high-risk patterns


### Recommendation:
Focus on early engagement, flexible pricing, and contract upgrades.

Customer satisfaction is more influenced by experience and cost than personal traits.
